## 임직원 수 및 급여

In [1]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
pd.options.display.float_format = '{:.6f}'.format

In [2]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [3]:
my_key = pd.read_csv('data/dart_key.txt', header = None)[0][0]

In [4]:
my_key# = '5f636989908de5019426008adba1a078e3c134f9'

'd6546705cd01f3257395ab1d45d6099f97d60e04'

In [5]:
base_url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.xml'
surv_key = '?crtfc_key=' + my_key
corp_code = '&corp_code=' + '00' + '365387'
bsns_year = '&bsns_year=' + '2024'
reprt_code = '&reprt_code=11011&fs_div=CFS'

In [6]:
url = base_url + surv_key + corp_code + bsns_year + reprt_code
url

'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.xml?crtfc_key=d6546705cd01f3257395ab1d45d6099f97d60e04&corp_code=00365387&bsns_year=2024&reprt_code=11011&fs_div=CFS'

In [7]:
response = requests.get(url)
response

<Response [200]>

In [8]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>000</status><message>정상</message><list><rcept_no>20250320000455</rcept_no><reprt_code>11011</reprt_code><bsns_year>2024</bsns_year><corp_code>00365387</corp_code><sj_div>BS</sj_div><sj_nm>재무상태표</sj_nm><account_id>ifrs-full_Assets</account_id><account_nm>자산총계</account_nm><account_detail>-</account_detail><thstrm_nm>제 25 기</thstrm_nm><thstrm_amount>1717770434389</thstrm_amount><frmtrm_nm>제 24 기</frmtrm_nm><frmtrm_amount>1622157341878</frmtrm_amount><bfefrmtrm_nm>제 23 기</bfefrmtrm_nm><bfefrmtrm_amount>1483368740285</bfefrmtrm_amount><ord>7</ord><currency>KRW</currency></list><list><rcept_no>20250320000455</rcept_no><reprt_code>11011</reprt_code><bsns_year>2024</bsns_year><corp_code>00365387</corp_code><sj_div>BS</sj_div><sj_nm>재무상태표</sj_nm><account_id>ifrs-full_CurrentAssets</account_id><account_nm>유동자산</account_nm><account_detail>-</account_detail><thstrm_nm>제 25 기</thstrm_nm><thstrm_amount>304434525487</thstrm_amoun

In [9]:
soup = BeautifulSoup(response.text, "lxml")
soup

C:\Users\USER\AppData\Local\Temp\ipykernel_1784\2611772931.py:1: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><result><status>000</status><message>정상</message><list><rcept_no>20250320000455</rcept_no><reprt_code>11011</reprt_code><bsns_year>2024</bsns_year><corp_code>00365387</corp_code><sj_div>BS</sj_div><sj_nm>재무상태표</sj_nm><account_id>ifrs-full_Assets</account_id><account_nm>자산총계</account_nm><account_detail>-</account_detail><thstrm_nm>제 25 기</thstrm_nm><thstrm_amount>1717770434389</thstrm_amount><frmtrm_nm>제 24 기</frmtrm_nm><frmtrm_amount>1622157341878</frmtrm_amount><bfefrmtrm_nm>제 23 기</bfefrmtrm_nm><bfefrmtrm_amount>1483368740285</bfefrmtrm_amount><ord>7</ord><currency>KRW</currency></list><list><rcept_no>20250320000455</rcept_no><reprt_code>11011</reprt_code><bsns_year>2024</bsns_year><corp_code>00365387</corp_code><sj_div>BS</sj_div><sj_nm>재무상태표</sj_nm><account_id>ifrs-full_CurrentAssets</account_id><account_nm>유동자산</account_nm><account_detail>-</account_detail><thstrm_nm>제 25 기</thstrm_nm><thstrm_amount>304434525487</t

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import re

def get_employee_status(cdf, my_key, start_year=2019, end_year=2024):
    base_url = 'https://opendart.fss.or.kr/api/empSttus.xml'
    reprt_code = '11011'  # 사업보고서

    # 1. 기업 코드 포맷팅 (8자리) 및 기본 골격 준비
    corp_map = cdf[['corp_code', 'corp_name']].copy()
    corp_map['corp_code'] = corp_map['corp_code'].astype(str).str.zfill(8)
    corp_codes = corp_map['corp_code']
    
    # 최종 결과를 담을 리스트
    final_year_dfs = []

    # ---------------- Helper Functions ----------------
    def parse_tenure(text):
        if not text or text == '-': return 0.0
        text = str(text).strip()
        year_match = re.search(r'(\d+)년', text)
        month_match = re.search(r'(\d+)개월', text)
        years = float(year_match.group(1)) if year_match else 0.0
        months = float(month_match.group(1)) if month_match else 0.0
        if years == 0 and months == 0:
            try:
                return float(re.sub(r'[^0-9.]', '', text))
            except:
                return 0.0
        return years + (months / 12.0)

    def clean_num(text):
        if not text or text == '-': return 0
        try:
            return float(str(text).replace(',', ''))
        except:
            return 0
    # --------------------------------------------------

    for year in range(start_year, end_year + 1):
        print(f"\n=== {year}년 임직원 정보 수집 중 ===")
        
        # 해당 연도의 수집된 데이터를 담을 리스트
        scraped_data = []

        for code in tqdm(corp_codes, desc=f"{year}년 기업 진행", ncols=100):
            params = {
                'crtfc_key': my_key,
                'corp_code': code,
                'bsns_year': str(year),
                'reprt_code': reprt_code
            }

            # --- API 호출 및 데이터 파싱 ---
            # 실패하더라도 여기서 continue로 건너뛰면 안됨 -> loop 끝에서 데이터 유무 확인
            
            row_data = None # 성공 시 데이터를 담을 변수

            try:
                r = requests.get(base_url, params=params, timeout=10)
                # r.raise_for_status() # 404 등이 떠도 멈추지 않고 넘어가도록 주석 처리 가능
                
                soup = BeautifulSoup(r.text, "lxml-xml")
                status = soup.find('status')
                
                # 데이터가 정상적으로 있는 경우에만 파싱 로직 실행
                if status and status.text == '000':
                    items = soup.find_all('list')
                    if items:
                        # 집계 변수 초기화
                        stats = {
                            'man': {'full': 0, 'part': 0, 'salary': 0, 'weighted_tenure': 0, 'count': 0},
                            'woman': {'full': 0, 'part': 0, 'salary': 0, 'weighted_tenure': 0, 'count': 0}
                        }
                        
                        valid_items_found = False

                        for item in items:
                            sexdstn = item.find('sexdstn').text.strip() if item.find('sexdstn') else ''
                            target = None
                            if '남' in sexdstn: target = stats['man']
                            elif '여' in sexdstn: target = stats['woman']
                            else: continue

                            valid_items_found = True
                            rgllbr_co = clean_num(item.find('rgllbr_co').text)
                            sm = clean_num(item.find('sm').text)
                            part_time = max(0, sm - rgllbr_co)
                            salary = clean_num(item.find('fyer_salary_totamt').text)
                            tenure = parse_tenure(item.find('avrg_cnwk_sdytrn').text)

                            target['full'] += rgllbr_co
                            target['part'] += part_time
                            target['count'] += sm
                            target['salary'] += salary
                            target['weighted_tenure'] += (tenure * sm)

                        if valid_items_found:
                            # 최종 계산
                            total_man_count = stats['man']['count']
                            total_woman_count = stats['woman']['count']
                            total_count = total_man_count + total_woman_count
                            
                            if total_count > 0:
                                salary_total_man = stats['man']['salary']
                                salary_total_woman = stats['woman']['salary']
                                salary_total = salary_total_man + salary_total_woman

                                salary_ave_man = salary_total_man / total_man_count if total_man_count > 0 else 0
                                salary_ave_woman = salary_total_woman / total_woman_count if total_woman_count > 0 else 0
                                salary_ave = salary_total / total_count if total_count > 0 else 0

                                total_weighted_tenure = stats['man']['weighted_tenure'] + stats['woman']['weighted_tenure']
                                ave_work = total_weighted_tenure / total_count if total_count > 0 else 0

                                row_data = {
                                    'corp_code': code,
                                    'year': year, # Merge를 위한 키
                                    'sex': 'Total',
                                    'full_time': stats['man']['full'] + stats['woman']['full'],
                                    'part_time': stats['man']['part'] + stats['woman']['part'],
                                    'salary_total': salary_total,
                                    'salary_total_man': salary_total_man,
                                    'salary_total_woman': salary_total_woman,
                                    'salary_ave': salary_ave,
                                    'salary_ave_man': salary_ave_man,
                                    'salary_ave_woman': salary_ave_woman,
                                    'ave_work': ave_work
                                }
            except Exception as e:
                # 에러 발생 시 row_data는 None 유지
                pass

            # 데이터가 있으면 리스트에 추가
            if row_data:
                scraped_data.append(row_data)
            
            time.sleep(0.05) # API 호출 제한

        # -------------------------------------------------------
        # [핵심 수정] 
        # 해당 연도의 모든 기업이 포함된 기본 틀(template)을 만들고, 
        # 수집된 데이터를 Left Merge하여 누락된 기업은 NaN으로 남김
        # -------------------------------------------------------
        
        # 1. 수집된 데이터 DF화
        if scraped_data:
            df_year_scraped = pd.DataFrame(scraped_data)
        else:
            # 하나도 수집 안 된 경우 빈 DF 생성 (칼럼은 맞춰줌)
            df_year_scraped = pd.DataFrame(columns=[
                'corp_code', 'year', 'sex', 'full_time', 'part_time', 
                'salary_total', 'salary_total_man', 'salary_total_woman', 
                'salary_ave', 'salary_ave_man', 'salary_ave_woman', 
                'ave_work'
            ])

        # 2. 해당 연도의 전체 기업 리스트 준비 (Template)
        df_template = corp_map.copy()
        df_template['year'] = year # 모든 행에 현재 연도 부여

        # 3. Left Merge (전체 기업 리스트 기준)
        # on=['corp_code', 'year']를 기준으로 붙임. 없는 기업은 NaN이 됨.
        df_year_final = df_template.merge(df_year_scraped, on=['corp_code', 'year'], how='left')
        
        # (옵션) NaN이어도 'sex' 칼럼은 'Total'로 채우고 싶다면 아래 주석 해제
        # df_year_final['sex'] = df_year_final['sex'].fillna('Total')

        final_year_dfs.append(df_year_final)

    # 전체 연도 병합
    df_result = pd.concat(final_year_dfs, ignore_index=True)

    # 칼럼 순서 정리 (corp_name 포함)
    target_cols = [
        'corp_code', 'corp_name', 'year', 'sex', 'full_time', 'part_time', 
        'salary_total', 'salary_total_man', 'salary_total_woman', 
        'salary_ave', 'salary_ave_man', 'salary_ave_woman', 
        'ave_work'
    ]
    
    # 결과 DF에 없는 칼럼(API 실패로 생성 안 된 경우 등)은 NaN으로 생성해줌
    for col in target_cols:
        if col not in df_result.columns:
            df_result[col] = pd.NA

    return df_result[target_cols]

In [19]:
adf_2019_2024 = get_employee_status(cdf, my_key, start_year=2019, end_year=2024)


=== 2019년 임직원 정보 수집 중 ===


2019년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [01:35<00:00,  7.63it/s]



=== 2020년 임직원 정보 수집 중 ===


2020년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [01:33<00:00,  7.82it/s]



=== 2021년 임직원 정보 수집 중 ===


...

2022년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [01:32<00:00,  7.95it/s]



=== 2023년 임직원 정보 수집 중 ===


2023년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [01:31<00:00,  7.96it/s]



=== 2024년 임직원 정보 수집 중 ===


2024년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [01:32<00:00,  7.90it/s]


In [20]:
adf_2019_2024

,corp_code,corp_name,year,sex,full_time,part_time,salary_total,salary_total_man,salary_total_woman,salary_ave,salary_ave_man,salary_ave_woman,ave_work
0,00365387,AJ네트웍스,2019,Total,513.000000,18.000000,30112992000.000000,23306579000.000000,6806413000.000000,56709966.101695,67555301.449275,36593618.279570,3.470998
1,00125080,AK홀딩스,2019,Total,19.000000,1.000000,1302000000.000000,1103000000.000000,199000000.000000,65100000.000000,68937500.000000,49750000.000000,2.300000
2,00219097,BGF,2019,Total,107.000000,6.000000,8246000000.000000,6864000000.000000,1382000000.000000,72973451.327434,82698795.180723,46066666.666667,6.938053
3,01263022,BGF리테일,2019,Total,2010.000000,631.000000,113452000000.000000,92536000000.000000,20916000000.000000,42957970.465733,47723568.849923,29794871.794872,1.900000
4,00858364,BNK금융지주,2019,Total,79.000000,17.000000,13196000000.000000,12567000000.000000,629000000.000000,137458333.333333,144448275.862069,69888888.888889,2.273438
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,00111421,휴니드테크놀러지스,2024,Total,350.000000,9.000000,26354828000.000000,23753408000.000000,2601420000.000000,73411777.158774,76871870.550162,52028400.000000,8.451532
4382,00362238,휴비스,2024,Total,646.000000,58.000000,55799000000.000000,49968000000.000000,5831000000.000000,79259943.181818,80593548.387097,69416666.666667,15.529545
4383,00156488,휴스틸,2024,Total,563.000000,22.000000,35118000000.000000,33171000000.000000,1947000000.000000,60030769.230769,61314232.902033,44250000.000000,4.225641
4384,00103176,흥국화재,2024,Total,864.000000,97.000000,77082382000.000000,52365382000.000000,24717000000.000000,80210595.213319,103899567.460317,54085339.168490,9.524454


In [21]:
adf_2019_2024.isnull().sum()

corp_code             0
corp_name             0
year                  0
sex                   6
full_time             6
part_time             6
salary_total          6
salary_total_man      6
salary_total_woman    6
salary_ave            6
salary_ave_man        6
salary_ave_woman      6
ave_work              6
dtype: int64

In [22]:
adf_2019_2024.to_csv('data/worker/worker_total.csv', float_format = '%.6f',index = False)